In [ ]:
class LazyRecord:
    def __init__(self):
      self.exists = 5
    
    def __getattr__(self,name):
      value = f'Value for {name}'
      setattr(self,name,value)
      return value

class LoggingLazyRecord(LazyRecord):
  def __getattr__(self, name):
    print(f'* calling __getattr__({name})')
    result = super().__getattr__(name)
    print(f'* Returning {result}')
    return result

data = LoggingLazyRecord()
print(data.__dict__)
print(hasattr(data,'foo'))
print(data.__dict__)
print(hasattr(data,'foo'))


In [ ]:
class Validatingrecord:
  def __init__(self):
    self.exists = 5
  
  def __getattribute__(self, name):
    print(f"* Called  __getattribute__ with {name}")
    try:
      value = super().__getattribute__(name)
      print(f'* Found {name !r}, returning {value !r}')
      return value
    except AttributeError:
      value = f'Value for {name}'
      print(f'* setting {name !r} to  {value !r}')
      setattr(self, name, value)
      return value
data = Validatingrecord()
# print(data.exists)
print(data.foo)
print(data.foo)
  

In [ ]:
class SavingRecord:
  def __setattr__(self, name, value):
    print(f'SavingRecord Called __setattr__ {name !r} to {value !r}')
    super().__setattr__(name, value)

class LoggingSavingRecord(SavingRecord):
  def __setattr__(self, name, value):
    print(f'LoggingSavingRecord Called __setattr__ {name !r} to {value !r}')
    return super().__setattr__(name, value)

data  = LoggingSavingRecord()
print(data.__dict__)
data.foo = 5
print(data.__dict__)
data.foo =7
print(data.__dict__)

In [ ]:
class BrokenDictionaryRecord:
  def __init__(self,data):
    self._data = data
  def __getattribute__(self, name):
     print(f"* Called  __getattribute__ with {name}")
     result = super().__getattribute__('_data')
     return result[name]

data = BrokenDictionaryRecord({'foo':3})
data.foo
    

In [ ]:
class Meta(type):
  def __new__(cls,name,basses,class_dict):
    print(f'Running {cls}.__new__ for {name}')
    print('Basses',basses)
    print(class_dict)
    return type.__new__(cls,name,basses,class_dict)
  
class Myclass(metaclass=Meta):
  print('test')
  stuff = 123
  def foo(self):
    pass


class Mysubclass(Myclass):
  print('test1')
  other = 567
  
  def bar(self):
    pass
    

In [1]:
class ValidatePolygon(type):
  def __new__(cls, name, bases, class_dict):
    if bases:
      if class_dict['sides'] < 3:
        raise ValueError('polyron need 3+ sides')
    return type.__new__(cls, name, bases, class_dict)

class polygon(metaclass=ValidatePolygon):
  
  sides = None
  @classmethod
  def interior_angles(cls):
    return (cls.sides - 2) * 180
  

class Triangle(polygon):
  sides = 3
  
class Rectangle(polygon):
  sides = 4
  
class Nonagon(polygon):
  sides = 9
  


assert Triangle.interior_angles() == 180
assert Rectangle.interior_angles() == 360
assert Nonagon.interior_angles() == 1260 

In [ ]:
class BetterPolygon:
  sides = None
  
  def __init_subclass__(cls):
    super().__init_subclass__()
    if cls.sides <3:
      raise ValueError('Polygons need 3+ sides')
  
  @classmethod
  def internal_angle(cls):
    return (cls.sides - 2) * 180

class Hexagon(BetterPolygon):
  sides = 6
  
assert Hexagon.internal_angle() == 120

In [ ]:
class ValidatePolygon(type):
  def __new__(cls, name, bases, class_dict):
    if bases:
      if class_dict['sides'] < 3:
        raise ValueError('polyron need 3+ sides')
    return type.__new__(cls, name, bases, class_dict)

class polygon(metaclass=ValidatePolygon):
  sides = None
  @classmethod
  def interior_angles(cls):
    return (cls.sides - 2) * 180
  
class ValidateFill(type):
  def __new__(cls, name, bases, attrs):
    if bases:
      if attrs['color'] not in ['red','green']:
        raise ValueError('Fill color must be supportesd')
    return type.__new__(cls, name, bases, attrs)

class Filled(metaclass=ValidateFill):
  color = None

class RedFilled(Filled,polygon):
  color = 'red'
  sides = 5


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [11]:
class ValidatePolygon(type):
  def __new__(cls, name, bases, class_dict):
    if not class_dict.get('is_root'):
      if class_dict['sides'] < 3:
        raise ValueError('polyron need 3+ sides')
    return type.__new__(cls, name, bases, class_dict)
  
class Polygon(metaclass=ValidatePolygon):
  
  is_root = True
  sides = None


class ValidateFilledPolygon(ValidatePolygon):
  def __new__(cls, name, bases, class_dict):
    if not class_dict.get('is_root'):
      if class_dict['color'] not in ('red','green'):
        raise ValueError('Fill color must be supported')
    return super().__new__(cls, name, bases, class_dict)

class FilledPolygon(Polygon,metaclass=ValidateFilledPolygon):
  is_root = True
  color = None
  
class GreenPentagon(FilledPolygon):
  color = 'green'
  sides = 5

greenie = GreenPentagon()
assert isinstance(greenie, Polygon)

class OrangePentagon(FilledPolygon):
  color = 'orange'
  sides = 5

ValueError: Fill color must be supported

In [ ]:
class BetterPolygon:
  sides = None
  
  def __init_subclass__(cls):
    super().__init_subclass__()
    if cls.sides <3:
      raise ValueError('Polygons need 3+ sides')
  
  @classmethod
  def internal_angle(cls):
    return (cls.sides - 2) * 180

class Filled:
  color = 'red'
  
  def __init_subclass__(cls):
    super().__init_subclass__()
    if cls.color not in ('red', 'blue', 'green'):
      raise ValueError('Invalid color')

class RedTriangle(Filled,BetterPolygon):
  color = 'red'
  sides = 3

ruddy = RedTriangle()
assert isinstance(ruddy, Filled)
assert isinstance(ruddy, BetterPolygon)

In [17]:

class Top:
  def __init_subclass__(cls):
    super().__init_subclass__()
    print(f'Top for {cls}')

class Left(Top):
  def __init_subclass__(cls):
    super().__init_subclass__()
    print(f'Left for {cls}')

class Right(Top):
  def __init_subclass__(cls):
    super().__init_subclass__()
    print(f'Right for {cls}')

class Bottom(Left,Right):
  def __init_subclass__(cls):
    super().__init_subclass__()
    print(f'Bottom for {cls}')
    
print(Bottom.__mro__)


Top for <class '__main__.Left'>
Top for <class '__main__.Right'>
Top for <class '__main__.Bottom'>
Right for <class '__main__.Bottom'>
Left for <class '__main__.Bottom'>
(<class '__main__.Bottom'>, <class '__main__.Left'>, <class '__main__.Right'>, <class '__main__.Top'>, <class 'object'>)


In [6]:
import json
class Serializable:
  def __init__(self,*args):
    self.args = args
  
  def serialize(self):
    return json.dumps({'args':self.args})

class Point2D(Serializable):
  def __init__(self,x,y):
    super().__init__(x,y)
    self.x = x
    self.y = y
  
  def __repr__(self):
    return f'Pont2D({self.x},{self.y})'


class Deserializable(Serializable):
  @classmethod
  def deserialize(cls,json_data):
    params = json.loads(json_data)
    return cls(*params['args'])
  
class BetterPont2D(Deserializable):
  def __init__(self,x,y):
    super().__init__(x, y)
    self.x = x
    self.y = y
  
  def __repr__(self):
    return f'Pont2D({self.x},{self.y})'


before = BetterPont2D(5,3)
print(before)
data = before.serialize()
print(data)
after = BetterPont2D.deserialize(data)
print(after)

Pont2D(5,3)
{"args": [5, 3]}
Pont2D(5,3)


In [13]:
class BetterSerializable:
  def __init__(self,*args):
    self.args = args
  
  def serialize(self):
    return json.dumps({
      "class": self.__class__.__name__,
      "args": self.args
    })
  
  def __repr__(self):
    name = self.__class__.__name__
    args_str = ','.join(str(x) for x in self.args)
    return f"{name}({args_str})"

registry = {}

def register_class(traget_class):
  registry[traget_class.__name__] = traget_class

def deserialize(data):
  params = json.loads(data)
  name = params['class']
  target_class = registry[name]
  return target_class(*params['args'])

class EvenBetterPoint2D(BetterSerializable):
    def __init__(self,x,y):
      super().__init__(x, y)
      self.x = x
      self.y = y
      
register_class(EvenBetterPoint2D)

before = EvenBetterPoint2D(1,2)
print(before)

data =before.serialize()
print(data)

after = deserialize(data)
print(after)

EvenBetterPoint2D(1,2)
{"class": "EvenBetterPoint2D", "args": [1, 2]}
EvenBetterPoint2D(1,2)


In [ ]:
# class Filed:
#   def __init__(self,name):
#     self.name = name
#     self.internal_name = '_' + self.name
  
#   def __get__(self,instance,instance_type):
#     if instance is None:
#       return self
#     return getattr(instance,self.internal_name,'')
  
#   def __set__(self,instance,value):
#     setattr(instance,self.internal_name,value)
    
# class Customer:
#   first_name = Filed('first_name')
#   last_name = Filed('last_name')
#   prefix  = Filed('prefix')
#   suffix = Filed('suffix')

# cust = Customer()
# print(f'{cust.first_name !r}{cust.__dict__}')
# print(f'{cust.first_name !r}{cust.__dict__}')
# cust.first_name = 'Joe'
# print(f'{cust.first_name !r}{cust.__dict__}')



class Meta(type):
  def __new__(meta,name,basses,attrs):
    print('attrs',attrs)
    for key,value in attrs.items():
      if isinstance(value,Filed):
        value.name = key
        value.internal_name = '_' + key
    cls = type.__new__(meta,name,basses,attrs)
    return cls

# class DatabaseRow(metaclass=Meta):
#   pass

class Filed:
  
  def __init__(self):
    self.name = None
    self.internal_name = None
    
  def __set__(self,instance,value):
    setattr(instance,self.internal_name,value)
    
  def __get__(self,instance,cls):
    if instance is None: return self
    return getattr(instance,self.internal_name,'')

class BetterCustomer(metaclass=Meta):
  first_name = Filed()
  last_name = Filed()
  prefix  = Filed()
  suffix = Filed()
  
cust = BetterCustomer()
print(f'{cust.first_name !r}{cust.__dict__}')
cust.first_name = 'Joe'
print(f'{cust.first_name !r}{cust.__dict__}')



meta <class '__main__.Meta'>
name BetterCustomer
basses ()
attrs {'__module__': '__main__', '__qualname__': 'BetterCustomer', 'first_name': <__main__.Filed object at 0x00000157DC2DB560>, 'last_name': <__main__.Filed object at 0x00000157DC2DAF30>, 'prefix': <__main__.Filed object at 0x00000157DC2D9550>, 'suffix': <__main__.Filed object at 0x00000157DC2DBAD0>}
''{}
'Joe'{'_first_name': 'Joe'}


In [ ]:
class Filed:
  
  def __init__(self):
    self.name = None
    self.internal_name = None
  
  def __set_name__(self,owenr,name):
    self.name = name
    self.internal_name = '_' + name
    
  def __set__(self,instance,value):
    setattr(instance,self.internal_name,value)
    
  def __get__(self,instance,cls):
    if instance is None: return self
    return getattr(instance,self.internal_name,'')

class FixedCustom:
  first_name = Filed()
  last_name = Filed()
  prefix  = Filed()
  suffix = Filed()

cust = FixedCustom()
print(f'{cust.first_name !r}{cust.__dict__}')
cust.first_name = 'Joe'
print(f'{cust.first_name !r}{cust.__dict__}')

  

<class '__main__.FixedCustom'>
<class '__main__.FixedCustom'>
<class '__main__.FixedCustom'>
<class '__main__.FixedCustom'>
''{}
'Joe'{'_first_name': 'Joe'}


In [3]:
def dump(**kwargs):
    return kwargs
dump(**{'x':1},y=2,**{'z':3})

{'x': 1, 'y': 2, 'z': 3}

In [ ]:
d1 = {'a':3,'b':4}
d2 = {'a':2,'b':4,'c':5}
d1|=d2


{'a': 2, 'b': 4, 'c': 5}